In [ ]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 39.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
cv_data= json.load(open('/content/drive/MyDrive/CV-Parsing-using-Spacy-3/data/training/train_data.json'))

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/drive/MyDrive/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        ents = []
        entity_indices = set()
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity:
                continue
            entity_indices.update(range(start, end))
            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue
            if span is None:
                err_data = str([start, end]) + "     " + str(text) + '\n'
                file.write(err_data)
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_data,test_size=0.1)


In [ ]:
len(train), len(test)

(180, 20)

In [ ]:
file= open('error.txt','w')
db= get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 20/20 [00:00<00:00, 77.19it/s]


In [ ]:
len(db.tokens)

20

In [ ]:
db.tokens

[array([[15118689538241911910, 10690858767396548707,                    0,
         ...,                    0,                    0,
                            0],
        [ 6361465612319394292,  5786846126360288792,                    0,
         ...,                    0,                    0,
                            0],
        [15777851466613025305, 16298582478581282910,                    0,
         ...,                    0,                    0,
                            0],
        ...,
        [ 2283656566040971221,  2283656566040971221,                    0,
         ...,                    0,                    0,
                            0],
        [ 6250203919658647129,  6250203919658647129,                    0,
         ...,                    0,                    0,
                            0],
        [13805906486645978168, 13805906486645978168,                    0,
         ...,                    0,                    0,
                            0

In [ ]:

!python -m spacy train /content/drive/MyDrive/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train=/content/train_data.spacy --paths.dev=/content/test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        4729.74   1574.43    0.00    0.00    0.00    0.00
  3     200      185737.32  66431.69   32.42   59.63   22.26    0.32
  6     400       37286.05  23092.02   44.76   73.44   32.19    0.45
  9     600       21342.27  20216.44   54.97   58.24   52.05    0.55
 

In [ ]:
import shutil

shutil.make_archive("/content/drive/MyDrive/CV-Parsing-using-Spacy-3/final", 'zip', "/content/output")

'/content/drive/MyDrive/CV-Parsing-using-Spacy-3/final.zip'

In [ ]:
nlp= spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
doc=nlp("""SUMEDH SHAH
(412) 954-8546

sumedhns@andrew.cmu.edu

https://www.linkedin.com/in/sumedhs/

EDUCATION
Carnegie Mellon University, Pittsburgh
Master of Information Systems Management - Business Intelligence and Data Analytics
December 2022
GPA- 3.96/4.00
Teaching Assistant- Unstructured Data Analytics, NoSQL Database Management
Maharashtra Institute of Technology, India
Bachelor of Engineering in Electronics and Telecommunication Engineering
June 2019
GPA- 7.79/10
SKILLS
 Languages: Java, Python (w/ framework – Pyspark), SQL, Bash (Linux), Scala
 Database and Big Data: SQL (Oracle, MySQL, Postgres), Hadoop (w/ Hive and Apache Spark), ETL Modeling, Apache Solr,
MapReduce, PyTorch, Tableau, Spacy, Redis, MongoDB, and Neo4j database
 Cloud: AWS Certified Solutions Architect – Associate, Microsoft Azure, Azure Data Factory, Databricks
WORK EXPERIENCE
PPG Industries, Pittsburgh
AI/ML Intern
May 2022- August 2022
 [azure pipelines] Designed and managed data flows and pipelines in Azure Data Factory for ingestion from relational databases,
REST APIs, Delta Lake, Sharepoint and other cloud file storage systems
 [automation] Automated the manual process of ingesting Design of Experiment data from workbooks in Sharepoint using
PySpark scripts written in Databricks, thereby reducing ingestion and curation time by 90%
Modak Analytics LLP, India
Data Engineer
September 2019- July 2021
 [data ingestion] Developed PySpark code that minimized ingestion time by 60% to consume terabytes of data from structured
and unstructured sources daily using automated bot workflows into the client data lake as Hive tables, leading to convenient
downstream analysis and data transformation (using SQL) applications
 [data curation] Built complex Spark SQL queries to curate pharmaceutical data on Dataframe collection, these were utilized
in data analysis and modelling for accelerating the timeline of the drug discovery process by 50%
 [pipeline automation and visualization] Conducted the compilation of Python workflows using Subprocess and OS modules
which executed Scala scripts to index curated data into Apache Solr and Neo4j graph database for utilization by pharmaceutical
SMEs; automated workflows would be triggered daily to bring in updated data
 [communication] Collaborated with clients and took the initiative to plan strategies to enhance the company’s native
platform and helped develop creative business requirements for a clinical trials project
 [leadership] Spearheaded a team of 5 members who collectively created an efficient data pipeline in PySpark for a
client use case that involved crawling, ingestion and transformation of terabytes of clinical trials data, which resulted
in the reduction of original processing time by 50%
ACADEMIC PROJECTS
Predicting Order Returns (https://github.com/Sumedh1197/Predicting-Order-Returns)
March 2022- May 2022



Predicting whether an online order will be returned to an online retailer as a step to increase net margins and improve supply
chain & inventory management; according to research this is soon expected to be a trillion-dollar problem
 Initially performing data preparation and feature engineering for downstream modelling and finally predicting the orders
which would be returned using Machine Learning classification algorithms such as Decision Tree, Random Forest, Logistic
Regression, Naive Bayes, and Perceptron
 Improvement of performance metrics mainly Recall since False Negatives, in this case, are a bigger concern for a retailer
[Python, Jupyter Notebook, Scikit-Learn, Numpy, Plotly, Seaborn]
EDA on H&M Transactions Data (https://github.com/Sumedh1197/EDA_Project)
January 2022- March 2022



Manufactured call-to-action (CTA’s) for H&M regarding production of category, colour, and pricing using EDA.
Associating the best channel of sales for the above actionable items. Presented the output in a time series format with CTAs
for each quarter
 Sample CTA Quarter 1 - Jan-March - H&M should focus on selling lower-priced products in the Ladieswear category in
Black, Blue and Pink colour and sell them through online channels
[Python, Jupyter Notebook, Altair, Pandas, Scipy, Numpy]
Other Projects: Travel Planner GUI, Recommendation System (https://github.com/Sumedh1197)
COMMUNITY SERVICE AND LEADERSHIP EXPERIENCE
 Live Life Love Life Charity Foundation- Fundraising Coordinator
October 2017- October 2019
Organized events to spread cancer awareness and raised approximately $40,000 from donations to provide medical assistance
for the underprivileged to fight breast cancer
 Thermax Foundation
June 2017- August 2017
Focused on educating underprivileged children in mathematics and science for entrance exams
""")

for ent in doc.ents:
  print(ent.text,"  ->>>>", ent.label_)

SUMEDH SHAH   ->>>> Name
412)   ->>>> Email Address
sumedhns@andrew.cmu.edu

   ->>>> Email Address
Carnegie Mellon University   ->>>> College Name
Master of Information Systems Management   ->>>> Degree
2022   ->>>> Graduation Year
Maharashtra Institute of Technology   ->>>> College Name
Bachelor of Engineering in Electronics and Telecommunication Engineering   ->>>> Degree
2019   ->>>> Graduation Year
 Languages: Java, Python (w/ framework – Pyspark), SQL, Bash (Linux), Scala
   ->>>> Skills
Database and Big Data: SQL (Oracle, MySQL, Postgres), Hadoop (w/ Hive and Apache Spark), ETL Modeling, Apache Solr,
MapReduce, PyTorch, Tableau, Spacy, Redis, MongoDB, and Neo4j database
 Cloud: AWS Certified Solutions Architect – Associate,   ->>>> Skills
Microsoft   ->>>> Companies worked at
COMMUNITY SERVICE AND LEADERSHIP EXPERIENCE   ->>>> Designation
